In [2]:
import boto3

In [3]:
session = boto3.Session()
credentials = session.get_credentials()

# Credentials are refreshable, so accessing your access key / secret key
# separately can lead to a race condition. Use this to get an actual matched
# set.
credentials = credentials.get_frozen_credentials()
access_key = credentials.access_key
secret_key = credentials.secret_key
region = session.get_available_regions("ec2")
session.region_name
#session.get_available_services()

'us-east-2'

In [4]:
ec2_client = boto3.client('ec2')
ec2_res = boto3.resource('ec2')

In [8]:
ec2_client.describe_instances()

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'ImageId': 'ami-0277b52859bac6f4b',
     'InstanceId': 'i-0da4f7758b48ce959',
     'InstanceType': 't2.micro',
     'KeyName': 'jr-keypair',
     'LaunchTime': datetime.datetime(2021, 7, 6, 17, 25, 41, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'us-east-2a',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': 'ip-172-31-0-132.us-east-2.compute.internal',
     'PrivateIpAddress': '172.31.0.132',
     'ProductCodes': [],
     'PublicDnsName': '',
     'State': {'Code': 80, 'Name': 'stopped'},
     'StateTransitionReason': 'User initiated (2021-07-06 18:23:34 GMT)',
     'SubnetId': 'subnet-0f4d9464',
     'VpcId': 'vpc-046cdd6f',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
       'Ebs': {'AttachTime': datetime.datetime(2021, 6, 29, 22, 38, 14, tzinfo=tzutc()),
        'DeleteOnTermination': T

In [27]:
def list_instances(ec2_res):
    for my_instance in ec2_res.instances.all():
        print(
         f"Id: {my_instance.id}\nPlatform: {my_instance.platform}\nType: {my_instance.instance_type}\nPublic IPv4: {my_instance.public_ip_address}\nAMI: {my_instance.image.id}\nState: {my_instance.state}\n"
     )

In [29]:
list_instances(ec2_res)

Id: i-0da4f7758b48ce959
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-09492a4aeb532a4f3
Platform: None
Type: t2.micro
Public IPv4: 13.58.248.59
AMI: ami-0277b52859bac6f4b
State: {'Code': 64, 'Name': 'stopping'}



In [6]:
def instance_state_checks(ec2_res,state):
    # create filter for instances in running state
    filters = [
        {
            'Name': 'instance-state-name', 
            'Values': [state]
        }
    ]
    
    # filter the instances
    instances = ec2_res.instances.filter(Filters=filters)

    RunningInstances = []

    for instance in instances:
        # for each instance, append to array and print instance id
        RunningInstances.append(instance.id)
        print(instance.id)

In [34]:
instance_state_checks(ec2_res,"stopped")

i-0da4f7758b48ce959
i-00f25c1aafffbf5dc


In [35]:
instance_state_checks(ec2_res,"running")

In [36]:
instance_state_checks(ec2_res,"terminated")

i-030d02cf812d65158


---

### Create and run new Instance

Using same Image ID for all Instances

In [9]:
new_instance = ec2_client.run_instances(ImageId="ami-0277b52859bac6f4b",
                                InstanceType='t2.micro',
                                MinCount=1,
                                MaxCount=1, 
                                InstanceInitiatedShutdownBehavior='terminate', 
                            )

In [11]:
pwd

'/Users/chowdahead/Documents/GitHub/aws-cloud-computing-python'

In [22]:
new_instance['Instances'][0]['InstanceId']

'i-09492a4aeb532a4f3'

In [15]:
dic = new_instance['Instances'][0]
type(dic)

dict

In [18]:
import json

with open("dict_to_json_textfile.json", 'w') as fout:
    json_dumps_str = json.dumps(dic, indent=4, sort_keys=True, default=str)
    print(json_dumps_str, file=fout)

In [23]:

with open('dict_to_json_textfile.json') as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(data['InstanceId'])

i-09492a4aeb532a4f3


In [42]:
list(new_instance.keys())
instances = new_instance['Instances'][0]

AMI = new_instance['Instances'][0]["ImageId"]
print(f'{AMI} {instances["InstanceId"]}')

ami-0277b52859bac6f4b i-0275ca887da5eb3a8


---

### Stop Intance(s)

In [28]:
ec2_client.stop_instances(InstanceIds=["i-09492a4aeb532a4f3"])

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'},
   'InstanceId': 'i-09492a4aeb532a4f3',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '8ded8776-6abd-4d82-91cf-b09fca325caa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8ded8776-6abd-4d82-91cf-b09fca325caa',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Thu, 08 Jul 2021 00:24:08 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

### Terminate Instance(s)

In [27]:
ec2_client.terminate_instances(InstanceIds=[instances["InstanceId"]])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-030d02cf812d65158',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '71e9c4ac-771f-46af-962e-eed5b2ffad94',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '71e9c4ac-771f-46af-962e-eed5b2ffad94',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'vary': 'accept-encoding',
   'date': 'Thu, 01 Jul 2021 15:31:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

---

### We can always envoke the ```list_instances``` function at any time to see the info on our instances

In [45]:
list_instances(ec2_res)

Id: i-0da4f7758b48ce959
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-00f25c1aafffbf5dc
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-030d02cf812d65158
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 48, 'Name': 'terminated'}

Id: i-0275ca887da5eb3a8
Platform: None
Type: t2.micro
Public IPv4: 3.142.43.44
AMI: ami-0277b52859bac6f4b
State: {'Code': 64, 'Name': 'stopping'}

